# **Land Reclamation**

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

pd.options.display.float_format = '{:,}'.format# show , for thousands
pd.set_option('display.max_columns', 1000)# show all columns

from pathlib import Path

In [11]:
Hopper_capacity = 2700 # m3
suction_diameter = 700 #mm
Power = 940 #kW dredge pump diesel engine
v_TSHD = 11.5 #knots 
v = 3 #m/s
fraction_sand = 0.35

prod_sand = -6500*suction_diameter/1000 + 1250*(suction_diameter/1000)*np.log(Power)
prod_cemented = -3300*suction_diameter/1000 + 600*(suction_diameter/1000)*np.log(Power)

print('Maximum hourly dry mass production (sand):', round(prod_sand), 'm3/h')
print('Maximum production per week (sand):', round(prod_sand*24*7), 'm3/week\n')
print('Maximum hourly dry mass production (cemented):', round(prod_cemented), 'm3/h')
print('Maximum production per week (cemented):', round(prod_cemented*24*7), 'm3/week')

Maximum hourly dry mass production (sand): 1440 m3/h
Maximum production per week (sand): 241944 m3/week

Maximum hourly dry mass production (cemented): 565 m3/h
Maximum production per week (cemented): 94965 m3/week


**Sand layer**

In [12]:
V_sand_basin = 461691
V_cemented_basin = 227923
V_sand_donor = 465228
Total_volume = V_sand_basin + V_cemented_basin + V_sand_donor

print(f'Total volume to be dredged: {Total_volume:,.0f} m³')
print(f'Total volume to be dredged (sand): {V_sand_donor:,.0f} m³')

Total volume to be dredged: 1,154,842 m³
Total volume to be dredged (sand): 465,228 m³


In [13]:
week_sand_basin = 1.9 #weeks
week_sand_cemented = 5.2 #weeks

In [14]:
Prod_pipe = v*0.25*np.pi*(suction_diameter/1000)**2
Load_time = Hopper_capacity/Prod_pipe/3600
print(f'Loading time to fill hopper: {round(Load_time*60,0)} minutes \n')
storage = Hopper_capacity # m3

V_cycle = Load_time * prod_sand
V_total = Hopper_capacity # volume of storage
V_water = V_total - V_cycle # volume of water

m_soil = V_cycle *2650 # total mass of soil (dry tons)
m_water = V_water*1025 # total mass of water
m_total = m_soil + m_water # total mass in hopper

rho_mixture = m_total/V_total # mixture density
print('The average mixture density is ', round(rho_mixture,0), 'kg/m3')

Loading time to fill hopper: 39.0 minutes 

The average mixture density is  1588.0 kg/m3


In [15]:
rho_mix = rho_mixture
rho_sea = 1025
rho_sand = 1900

sand_fraction = 0.35
settlement_factor = 0.25
trapping_factor = 0.05
overflow_duration = 45 # mins
max_density = 2200
found = False

for overflow_duration_test in range(0, 46):  # 0 to 45 minutes
    solid_percentage = (rho_mix - rho_sea) / (rho_sand - rho_sea)
    mass_dry = prod_sand * rho_sand
    mass_mix = mass_dry / solid_percentage
    volume_mix = mass_mix / rho_mix

    mass_fines_mix = mass_mix * sand_fraction
    mass_fines_stirred = mass_fines_mix * 0.03
    mass_fines_settle = mass_fines_mix - mass_fines_stirred
    mass_overflow = ((Load_time * 60 + overflow_duration_test) / (Load_time * 60)) * (1 - settlement_factor) * (1 - trapping_factor) * mass_fines_settle
    rho_overflow = ((mass_mix - mass_overflow) / mass_mix) * rho_sea + (mass_overflow / mass_mix) * rho_sand

    # stop if density reaches or exceeds max_density (including at 45 min)
    if rho_overflow >= max_density:
        print(f"Maximum allowed overflow duration: {overflow_duration_test} min")
        print(f"Density of overflow water: {rho_overflow:.1f} kg/m³")
        found = True
        break

    if not found:
     # did not reach max density within 0-45 min -> force use of 45 min
        overflow_duration_test = 45
print("Overflow density did not reach 2200 kg/m³ within 45 minutes. Using 45 min as overflow duration.")
print(f"Density of overflow water at 45 min: {rho_overflow:.1f} kg/m³")

t_loading = Load_time*60 + (overflow_duration_test) # total cycle time (min)
print(f"Loading time without overflow: {Load_time*60:.0f} mins")
print(f"Total time loading: {t_loading:.0f} mins")

# 🔹 Sand volume and mass per cycle
solid_fraction = (rho_mix - rho_sea) / (rho_sand - rho_sea)
sand_volume_cycle = Hopper_capacity * solid_fraction
sand_mass_cycle = sand_volume_cycle * rho_sand

print(f"Sand volume per cycle: {sand_volume_cycle:.1f} m³")
print(f"Sand mass per cycle: {sand_mass_cycle/1000:.1f} tonnes")


Overflow density did not reach 2200 kg/m³ within 45 minutes. Using 45 min as overflow duration.
Density of overflow water at 45 min: 1481.0 kg/m³
Loading time without overflow: 39 mins
Total time loading: 84 mins
Sand volume per cycle: 1737.4 m³
Sand mass per cycle: 3301.1 tonnes


In [16]:
offshore_distance = 7000 #km
Qavg = 3600 #m3/hr
v_TSHD = 11.5*0.514*60 #convert to m/minute
t_sailing = offshore_distance/v_TSHD #in minutes
t_rainbow = Hopper_capacity/Qavg*60 #in minutes
t_maneuvering = 15
t_cycle = t_loading + t_sailing + t_rainbow + t_sailing*0.9 +t_maneuvering
print(f"Total cycle time: {t_cycle:.1f} mins")
print(f"Sailing time: {t_sailing:.0f} mins")
print(f"Discharge time: {t_rainbow:.1f} mins")
print(f"Maneuvering time: {t_maneuvering:.1f} mins")


Total cycle time: 181.5 mins
Sailing time: 20 mins
Discharge time: 45.0 mins
Maneuvering time: 15.0 mins


In [17]:
n_cycles = V_sand_donor/sand_volume_cycle
total_cycle_minutes = n_cycles * t_cycle
total_working_hours = total_cycle_minutes / 60
total_days = total_working_hours / 22/0.85
total_weeks = total_days / 7
print(f'n_cylcles: {n_cycles:.0f} cycles')
print(f'Total days required to perform offshore dredging: {total_days:.1f} days')
print(f'Total weeks required to perform offshore dredging: {total_weeks/0.9:.1f} weeks')

n_cylcles: 268 cycles
Total days required to perform offshore dredging: 43.3 days
Total weeks required to perform offshore dredging: 6.9 weeks
